In [1]:
import re 
import os
import os.path as osp
import torch
import pandas
import pandas as pd
import json
from typing import Union
from typing import List
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from datasets import concatenate_datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

bin C:\Users\any\anaconda3\envs\oosij\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [2]:
c_path = './data/classification/ukairia777/finance_data.csv' # 분류 .csv
s_path = './data/summarization/aihub_news' # 요약 폴더내.json 들
template_path = './templates/multi.json' # 템플릿  .json

# The instruction dataset to use
dataset_name = "Smoked-Salmon-s/empathetic_dialogues_ko" # 싱글/멀티 대화형 데이터셋  허깅페이스 :

# The model that you want to train from the Hugging Face hub
model_name = "beomi/llama-2-koen-13b"

# Fine-tuned model name
new_model = "llama-2-ko-13b-ft-multi-task-t3_l2048_v2" # v1은 그대로, v2는 데이터 비율 조정 

In [4]:
dataset =  load_dataset(dataset_name) # train_args.data_path
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [6]:
c_dataset= task_classification_dataset(c_path)
s_dataset = task_summarization_dataset(s_path)
chat_dataset = chat_load_and_preprocess_data(dataset, tokenizer)

Map:   0%|          | 0/4846 [00:00<?, ? examples/s]

Map:   0%|          | 0/4846 [00:00<?, ? examples/s]

Map:   0%|          | 0/9836 [00:00<?, ? examples/s]

Map:   0%|          | 0/9836 [00:00<?, ? examples/s]

In [7]:
import time

start_time = time.time()

# 'output' 값에 대한 각 인덱스 구하기
positive_indices = [idx for idx, label in enumerate(c_dataset['output']) if label == '긍정']
neutral_indices = [idx for idx, label in enumerate(c_dataset['output']) if label == '중립']
negative_indices = [idx for idx, label in enumerate(c_dataset['output']) if label == '부정']

# 각 'output' 값에 대한 필요한 수의 샘플 선택
positive_train_size = 1263
neutral_train_size = 2296
negative_train_size = 504

positive_test_size = len(positive_indices) - positive_train_size
neutral_test_size = len(neutral_indices) - neutral_train_size
negative_test_size = len(negative_indices) - negative_train_size

# 훈련 데이터 인덱스 선택
positive_train_indices = positive_indices[:positive_train_size]
neutral_train_indices = neutral_indices[:neutral_train_size]
negative_train_indices = negative_indices[:negative_train_size]

# 테스트 데이터 인덱스 선택
positive_test_indices = positive_indices[positive_train_size:]
neutral_test_indices = neutral_indices[neutral_train_size:]
negative_test_indices = negative_indices[negative_train_size:]

# 훈련 데이터셋 및 테스트 데이터셋 생성
train_dataset_indices = positive_train_indices + neutral_train_indices + negative_train_indices
test_dataset_indices = positive_test_indices + neutral_test_indices + negative_test_indices

train_dataset = Dataset.from_dict({key: [c_dataset[key][idx] for idx in train_dataset_indices] for key in c_dataset.features})
test_dataset = Dataset.from_dict({key: [c_dataset[key][idx] for idx in test_dataset_indices] for key in c_dataset.features})

# 결과 확인
print("Train set shape:", len(train_dataset))
print("Test set shape:", len(test_dataset))

end_time = time.time()
elapsed_time = end_time - start_time
print("프로그램 실행 시간: ", elapsed_time, "초")

c_dataset = train_dataset

Train set shape: 4063
Test set shape: 783
프로그램 실행 시간:  198.401535987854 초


In [8]:
# 데이터셋의 크기
total_samples = len(s_dataset)

# 훈련 데이터의 크기
train_size = 4492

# 훈련 데이터 인덱스 범위 선택
train_indices = list(range(train_size))

# 테스트 데이터 인덱스 범위 선택 (나머지)
test_indices = list(range(train_size, total_samples))

# 훈련 데이터와 테스트 데이터로 분할
train_dataset = s_dataset.select(train_indices)
test_dataset = s_dataset.select(test_indices)

# 데이터셋 형태 확인
print("Train set shape:", len(train_dataset))
print("Test set shape:", len(test_dataset))

s_dataset = train_dataset

Train set shape: 4492
Test set shape: 5344


In [9]:
start_time = time.time()

# 각 타입에 해당하는 인덱스를 구합니다.
single_indices = [idx for idx, label in enumerate(chat_dataset['type']) if label == 'single']
multi2_indices = [idx for idx, label in enumerate(chat_dataset['type']) if label == 'multi_2']
multi3_indices = [idx for idx, label in enumerate(chat_dataset['type']) if label == 'multi_3']

# 각 타입에 대해 조정할 샘플 수를 정의합니다.
single_samples = 6094
multi2_samples = 3712
multi3_samples = 10756

# 샘플 인덱스를 선택합니다.
selected_indices = (single_indices[:single_samples] + 
                    multi2_indices[:multi2_samples] + 
                    multi3_indices[:multi3_samples])

# 선택된 인덱스를 제외한 나머지를 테스트용으로 사용합니다.
test_indices = set(range(len(chat_dataset))) - set(selected_indices)

# train 및 test 데이터셋을 생성합니다.
train_dataset = chat_dataset.select(selected_indices)
test_dataset = chat_dataset.select(list(test_indices))

# 결과 확인
print("Train set shape:", len(train_dataset))
print("Test set shape:", len(test_dataset))

end_time = time.time()
elapsed_time = end_time - start_time
print("프로그램 실행 시간: ", elapsed_time, "초")

chat_dataset = train_dataset

Train set shape: 20562
Test set shape: 6100
프로그램 실행 시간:  0.2825772762298584 초


In [10]:
task_dataset = task_combined_dataset(c_dataset, s_dataset)
task_dataset = DatasetDict({'train': task_dataset})
task_dataset = task_load_and_preprocess_data(task_dataset, tokenizer)
# 두 데이터셋 합치기
combined_dataset = concatenate_datasets([chat_dataset,task_dataset])
dataset = combined_dataset.shuffle(seed=42)  # seed 값은 원하는 값으로 변경 가능
dataset = dataset.shuffle(seed=42)  # seed 값은 원하는 값으로 변경 가능

Map:   0%|          | 0/8555 [00:00<?, ? examples/s]

In [12]:
print(dataset)
dataset['text'][2000]

Dataset({
    features: ['instruction', 'output', 'source', 'type', 'text'],
    num_rows: 29117
})


'이전 대화와 현재 대화의 명령어를 참고하여 가장 적절한 응답을 생성해주세요. 일상적인 대화의 경우, 응답 마지막에는 지금까지의 내용과 관련된 질문을 해주세요.\n\n[이전 대화]\n### 명령어: 최근에 스트레스 때문에 잠을 잘 못 자고 있어요. 무슨 조언이 있을까요?\n### 응답: 그게 정말 힘들겠네요. 스트레스로 인해 잠을 못자는 건 정말 힘든 일이에요. 잠을 잘 자려면 일상 생활에서 스트레스를 푸는 시간을 갖는 것이 중요해요. 한번 시도해 볼 만한 확실한 방법이 있을까요?\n### 명령어: 저는 스트레스 받을 때 운동을 하면 마음이 편해지는 것 같아요. 다른 방법도 있을까요?\n### 응답: 그렇네요, 운동은 정말 효과적인 스트레스 해소 방법 중 하나에요. 그 외에도 명상을 해보거나 취미 생활을 즐기는 것도 좋은 방법이에요. 어떤 취미가 있나요? 그 취미를 통해 스트레스를 해소해 볼 수도 있을 것 같아요.\n\n\n[현재 대화]\n### 명령어:\n취미는 없지만, 명상은 해보고 싶은데 어떻게 시작해야 할까요?\n\n### 응답:\n취미가 없으시다면 명상을 시작하는 것도 좋은 선택이에요. 명상은 마음에 평화를 가져다 주고 스트레스를 해소해 줄 수 있어요. 인터넷에 명상 가이드나 앱이 많이 있어요. 그 중에 한 가지를 선택해서 따라해보는 것도 좋을 것 같아요. 시작하시면 어떨까요?<|endoftext|>'

In [13]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1


################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [14]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 2

# Batch size per GPU for evaluation
per_device_eval_batch_size = 2

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 40000

# Log every X updates steps
logging_steps = 2500

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 2048 #None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
#device_map = {"": 0}
device_map = 'auto'

In [15]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [16]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [17]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [18]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [19]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

C:\Users\any\anaconda3\envs\oosij\lib\site-packages\peft\utils\other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/29117 [00:00<?, ? examples/s]

In [20]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
C:\Users\any\anaconda3\envs\oosij\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
2500,1.044700
5000,0.985800
7500,0.964700
10000,0.949900
12500,0.950300
15000,0.932000
17500,0.893100
20000,0.891100
22500,0.882700
25000,0.886800


C:\Users\any\anaconda3\envs\oosij\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=43677, training_loss=0.8959474886533888, metrics={'train_runtime': 386736.6422, 'train_samples_per_second': 0.226, 'train_steps_per_second': 0.113, 'total_flos': 1.968589823748096e+18, 'train_loss': 0.8959474886533888, 'epoch': 3.0})

In [21]:
save_path = 'models/' +  new_model

# Save trained model / tokenizer
trainer.model.save_pretrained(save_path)
trainer.tokenizer.save_pretrained(save_path)

('models/llama-2-ko-13b-ft-multi-task-t3_l2048_v2\\tokenizer_config.json',
 'models/llama-2-ko-13b-ft-multi-task-t3_l2048_v2\\special_tokens_map.json',
 'models/llama-2-ko-13b-ft-multi-task-t3_l2048_v2\\tokenizer.json')

In [3]:
def inference_output(prompt):
    # 인퍼런스 커스텀
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda") # return_token_type_ids=False
    # peft 일시, **tokenizer(prompt, return_tensors="pt", add_special_tokens=False) 아래 inputs['input_ids'] 교체 , 위는 삭제 
    output = model.generate(
        inputs["input_ids"],
        max_new_tokens = 256,
        do_sample=True,
        top_p=0.9,
        temperature=0.1,
        early_stopping=True,
        eos_token_id= 46332,
    )
    output = output[0].to("cpu")
    return tokenizer.decode(output)

In [4]:
save_path = 'models/' +  new_model
save_path

'models/llama-2-ko-13b-ft-multi-task-t3_l2048_v2'

In [5]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map= 'auto',
)
model = PeftModel.from_pretrained(base_model, save_path)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
import json

with open('./templates/template_v002.json', 'r',  encoding='utf-8') as file:
    data = json.load(file)

In [ ]:
his_log = ''
cnt = 0 
print('무엇이든 질문해보세요! ')
while True:
    query = input('[### YOU]: ')
    prompt = data['prompt'].format(history=his_log, instruction=query)
    result = inference_output(prompt)
    response = result.split('\n\n'+data['response_split'])[1]
    response = response.replace('<|endoftext|>', '').strip()
    print()
    print('[### Chatbot]: ',response)
    print()
    if cnt == 0:
        his_log = query  +'\n'+ response
    else:
        his_log = his_log +'\n'+query + '\n' + response
    cnt += 1 
    #end = int(input('끝낼꺼면 0 입력'))
    #if end == 0:
    if cnt == 10:
        print('대화종료')
        break

무엇이든 질문해보세요! 


[### YOU]:  안녕?



[### Chatbot]:  안녕하세요! 반갑습니다. 😊 오늘 하루는 어떻게 시작하셨나요?



[### YOU]:   오늘 하루는 기분 좋게 시작했어..



[### Chatbot]:  기분 좋게 시작하셨다니 저도 기분이 좋아지네요! 😊 오늘 하루 어떤 일을 계획하셨나요?



...


[### Chatbot]:  긍정



[### YOU]:   흠... 저칼로리도 중요하지. 유산소 운동까지 해서 꼭 정상 체중에 도달해야겠어.



[### Chatbot]:  그렇군요, 건강한 식습관과 유산소 운동까지 병행하신다니 정말 멋지세요! 👏👏 건강한 식습관과 유산소 운동이 다이어트에 어떤 영향을 미치는지 알고 계신가요?



[### YOU]:  그래 시간이 없으니, 대화는 이쯤 하자. 나가볼테니 더 안물어봐도 돼



[### Chatbot]:  네, 알겠습니다. 건강하고 행복한 하루 보내세요! 😊



In [ ]:
his_log = ''
cnt = 0 
print('무엇이든 질문해보세요! ')
while True:
    query = input('[### YOU]: ')
    prompt = data['prompt'].format(history=his_log, instruction=query)
    result = inference_output(prompt)
    response = result.split('\n\n'+data['response_split'])[1]
    response = response.replace('<|endoftext|>', '').strip()
    print()
    print('[### Chatbot]: ',response)
    print()
    if cnt == 0:
        his_log = query  +'\n'+ response
    else:
        his_log = his_log +'\n'+query + '\n' + response
    cnt += 1 
    #end = int(input('끝낼꺼면 0 입력'))
    #if end == 0:
    if cnt == 10:
        print('대화종료')
        break

무엇이든 질문해보세요! 


[### YOU]:  안녕하세요...



[### Chatbot]:  안녕하세요! 반갑습니다. 오늘 하루는 어떠셨나요?



[### YOU]:  잠을 잘못 자서 그런지 졸립고 피곤해서 시작부터 힘드네요.



[### Chatbot]:  잠을 잘못 자서 피곤하다니, 시작부터 힘드시겠어요. 😢 잠을 잘 못 자는 것은 정말 힘든 일이죠. 어떤 이유로 잠을 잘못 잤는지 말씀해주실 수 있을까요?



...


[### Chatbot]:  긍정



[### YOU]:  좋아하는 감귤도 먹기 힘들겠네요... 그래도 열심히 살아야죠.



[### Chatbot]:  맞아요, 물가가 상승하면 좋아하는 과일조차도 부담스러워지는군요. 😢 그럼에도 불구하고 열심히 살아야 한다는 마음가짐, 정말 멋지십니다. 감귤 외에도 좋아하는 과일이 있나요?



[### YOU]:  이만, 대화는 종료해주세요. 



[### Chatbot]:  네, 알겠습니다. 😊 좋은 하루 보내세요!



In [4]:
def task_classification_dataset(c_path):
  # Read the CSV file into a pandas DataFrame
  df = pd.read_csv(c_path)
  # Replace values in the 'labels' column
  df['labels'] = df['labels'].replace({'neutral': '중립', 'negative': '부정', 'positive': '긍정'})

  ## 허깅페이스 데이터셋화

  # Hugging Face의 dataset 포맷으로 변환
  huggingface_dataset = Dataset.from_pandas(df.rename(columns={'labels': 'output', 'kor_sentence': 'input'}))

  # instruction 추가 및 컬럼 순서 변경
  huggingface_dataset = huggingface_dataset.map(
      lambda example: {"instruction": '밑의 내용의 감성을 분석하고, 그것이 긍정, 중립, 아니면 부정인지 결정하고 대답해주세요. 해당 감정 레이블은 "긍정", "중립" 또는 "부정"입니다.',
                       "input": example["input"],
                       "output": example["output"]
                       },
      remove_columns=['sentence']
  )

  # 필드와 값을 추가
  sentiment_dataset = huggingface_dataset.map(
      lambda example: {'instruction': example['instruction'], 'input': example['input'], 'output': example['output'], 'source': 'github/ukairia777', 'type': 'task_classification'})

  return sentiment_dataset

def task_summarization_dataset(s_path):
  folder_path  = s_path

  # Initialize an empty list to store data
  data_list = []

  # Get a list of all files in the folder
  file_list = [f for f in os.listdir(folder_path) if f.endswith('.json')]

  # Loop through each JSON file and extract 'passage' and 'summary2'
  for file_name in file_list:
      file_path = os.path.join(folder_path, file_name)

      # Read the JSON file
      with open(file_path, 'r', encoding='utf-8') as file:
          data = json.load(file)

      # Extract 'passage' and 'summary2' fields
      passage = data.get('Meta(Refine)', {}).get('passage', '')
      summary2 = data.get('Annotation', {}).get('summary2', '')

      #if len(summary2.split('. ')) <= 1: # 원인 불명 ?  그냥 문장 길이 미만인 거만 짜르도록
      if len(summary2) <= 100: # 문장 길이가 100 이하인 것은 버리기
        continue

      # Append data to the list
      data_list.append({'file_name': file_name, 'body': passage, 'summary': summary2})

  # Convert the list of dictionaries to a pandas DataFrame
  df = pd.DataFrame(data_list)

  # Hugging Face의 dataset으로 변환
  hf_dataset = Dataset.from_pandas(df)

  # 필요한 형태로 변환
  hf_dataset = hf_dataset.map(
      lambda example: {'instruction': '밑의 내용을 요약해주세요.',
                       'input': example['body'],
                       'output': example['summary']
                       },
      remove_columns=['file_name','body','summary']
                     )
  # 필드와 값을 추가
  summary_dataset = hf_dataset.map(
      lambda example: {'instruction': example['instruction'], 'input': example['input'], 'output': example['output'], 'source': 'aihub', 'type': 'task_summarization'})
  return summary_dataset


def task_combined_dataset(c_dataset, s_dataset):
  # 두 데이터셋 합치기
  combined_dataset = concatenate_datasets([c_dataset, s_dataset])
  # 데이터셋 섞기
  combined_dataset = combined_dataset.shuffle(seed=42)  # seed 값은 원하는 값으로 변경 가능
  # instruction과 input을 합쳐서 새로운 instruction에 값을 주기
  combined_dataset = combined_dataset.map(lambda example: {'new_instruction': example['instruction'].replace('#','').strip() + '\n\n' + example['input'].rstrip(), 'output': example['output'], 'source': example['source'], 'type': example['type']})

  # 기존의 instruction과 input 삭제
  combined_dataset = combined_dataset.remove_columns(['instruction', 'input'])
  # 'new_instruction' 필드의 이름을 'instruction'으로 변경
  combined_dataset = combined_dataset.rename_column('new_instruction', 'instruction')

  return combined_dataset


class Chat_prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            template_name =  'template_v002' # "multi"
        file_name = osp.join("templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name, encoding='utf-8') as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )
    def generate_prompt(
        self,
        instruction: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from optional input
        # if a label (=response, =output) is provided, it's also appended.

        # 멀티 턴 대화 처리하는 부분
        def converter(sentence):
            result = re.sub(r"질문\s*", "### 명령어", sentence)
            result = re.sub(r"답변\s*", "### 응답", result)

            return result

        instruction = converter(instruction)
        new_instruction = instruction.split('\n')[-1]
        history = instruction[:-len(new_instruction)]
        try:
            new_instruction = new_instruction.split('### 명령어: ')[1]
        except:
            new_instruction = new_instruction.split('### 명령어: ')[0]

        res = self.template["prompt"].format(history=history, instruction=new_instruction)

        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

def chat_load_and_preprocess_data(dataset: dataset, tokenizer: tokenizer):
    data = dataset
    prompter = Chat_prompter(template_name = 'template_v002',
                        verbose = False)
    # prompter.template =>
    def generate_and_tokenize_prompt(instruction, output):
        full_prompt = prompter.generate_prompt(
            instruction = instruction,
            label = output,
        )
        #full_prompt_add_eos_token = '<s>\n' + full_prompt + '\n</s>'
        #full_prompt_add_eos_token = full_prompt + '</s>'
        full_prompt_add_eos_token = full_prompt + '<|endoftext|>'
        return full_prompt_add_eos_token

    original_dataset = data['train']
    # 새로운 데이터셋 생성
    new_dataset = {
      'instruction': original_dataset['instruction'],
      'output': original_dataset['output'],
      'source': original_dataset['source'],
      'type': original_dataset['type'],
      'text': [generate_and_tokenize_prompt(i,o) for i, o in zip(original_dataset['instruction'], original_dataset['output'])]
    }

    # 최종 데이터셋 생성
    final_dataset = Dataset.from_dict(new_dataset)

    return final_dataset

class Task_prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            template_name =  'template_v002' # "multi"
        file_name = osp.join("templates", f"{template_name}.json")
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name, encoding='utf-8') as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )
    def generate_prompt(
        self,
        instruction: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from optional input
        # if a label (=response, =output) is provided, it's also appended.

        # 멀티 턴 대화 처리하는 부분
        def converter(sentence):
            result = re.sub(r"질문\s*", "### 명령어", sentence)
            result = re.sub(r"답변\s*", "### 응답", result)

            return result

        instruction = converter(instruction)
        history = ''

        res = self.template["prompt"].format(history=history, instruction=instruction)

        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

def task_load_and_preprocess_data(dataset: dataset, tokenizer: tokenizer):
    data = dataset
    prompter = Task_prompter(template_name = 'template_v002',
                        verbose = False)
    # prompter.template =>
    def generate_and_tokenize_prompt(instruction, output):
        full_prompt = prompter.generate_prompt(
            instruction = instruction,
            label = output,
        )

        full_prompt_add_eos_token = full_prompt + '<|endoftext|>'
        return full_prompt_add_eos_token

    original_dataset = data['train']
    # 새로운 데이터셋 생성
    new_dataset = {
      'instruction': original_dataset['instruction'],
      'output': original_dataset['output'],
      'source': original_dataset['source'],
      'type': original_dataset['type'],
      'text': [generate_and_tokenize_prompt(i,o) for i, o in zip(original_dataset['instruction'], original_dataset['output'])]
    }

    # 최종 데이터셋 생성
    final_dataset = Dataset.from_dict(new_dataset)

    return final_dataset